In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/local/spark'

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
try:
    spark.sparkContext.stop()  # zamyka SparkSession, tylko wtedy mozna zmienic parametry mongodb
except NameError:
    print("")
spark = (SparkSession.builder.appName("projekt")
         .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector:10.0.5")
         .config("spark.mongodb.uri", "mongodb://127.0.0.1:27017/")
         .config("spark.mongodb.database", "projekt")
         .config("spark.mongodb.collection", "myNewCollection1")
         .set("spark.sql.legacy.parquet.int96RebaseModeInRead", "CORRECTED")
         .set("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
         .set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
         .set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")
         .enableHiveSupport()
         .getOrCreate())

23/01/08 00:13:55 WARN util.Utils: Your hostname, node1 resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s3)
23/01/08 00:13:55 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vagrant/.ivy2/cache
The jars for the packages stored in: /home/vagrant/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-93096cf4-1d2d-402b-aa53-a7d7ed65d9a1;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.5 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 2894ms :: artifacts dl 8ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-sync;4.5.1 from central in [default]
	org.mongodb.spark#mongo-spark-connector;10.0.5 from central in [default]
	---------------------------------------------------------------------
	|                  |

In [4]:
df = spark.read.format("mongodb").load()

In [5]:
df.show()

+--------------------+---+
|                 _id|  x|
+--------------------+---+
|63ba089c029e8a4bf...|1.0|
+--------------------+---+



In [ ]:
people.write.format("mongodb").mode("append").save()

## Czytanie danych do agregacji

In [7]:
#listowanie baz danych
spark.catalog.listTables()

[Table(name='employees', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='external_table_condo', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='external_table_nypd', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='salaries', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='wifi', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [10]:
#wczytanie danych nydp
spark.sql("REFRESH TABLE external_table_nypd") 
df_nypd = spark.table("external_table_nypd")
#braki danych usuniete w preprocessingu
df_nypd.printSchema()

root
 |-- cmplnt_num: integer (nullable = true)
 |-- cmplnt_fr_ts: timestamp (nullable = true)
 |-- cmplnt_to_ts: timestamp (nullable = true)
 |-- addr_pct_cd: integer (nullable = true)
 |-- rpt_dt: date (nullable = true)
 |-- ky_cd: integer (nullable = true)
 |-- ofns_desc: string (nullable = true)
 |-- pd_cd: integer (nullable = true)
 |-- pd_desc: string (nullable = true)
 |-- crm_atpt_cptd_cd: string (nullable = true)
 |-- law_cat_cd: string (nullable = true)
 |-- boro_nm: string (nullable = true)
 |-- loc_of_occur_desc: string (nullable = true)
 |-- prem_typ_desc: string (nullable = true)
 |-- juris_desc: string (nullable = true)
 |-- jurisdiction_code: integer (nullable = true)
 |-- parks_nm: string (nullable = true)
 |-- hadevelopt: string (nullable = true)
 |-- housing_psa: string (nullable = true)
 |-- x_coord_cd: integer (nullable = true)
 |-- y_coord_cd: integer (nullable = true)
 |-- susp_age_group: string (nullable = true)
 |-- susp_race: string (nullable = true)
 |-- susp

In [122]:
#wczytanie danych condo
spark.sql("REFRESH TABLE external_table_condo")  # refreshes table after adding files
df_condo = spark.table("external_table_condo")
#brak braków danych
df_condo.printSchema()

root
 |-- condo_section: string (nullable = true)
 |-- boro_block_lot: string (nullable = true)
 |-- address: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- building_classification: string (nullable = true)
 |-- total_units: integer (nullable = true)
 |-- year_built: integer (nullable = true)
 |-- gross_sqft: integer (nullable = true)
 |-- estimated_gross_income: integer (nullable = true)
 |-- gross_income_per_sqft: double (nullable = true)
 |-- estimated_expense: integer (nullable = true)
 |-- expense_per_sqft: double (nullable = true)
 |-- net_operating_income: integer (nullable = true)
 |-- full_market_value: integer (nullable = true)
 |-- market_value_per_sqft: double (nullable = true)
 |-- report_year: integer (nullable = true)



## Wstępna analiza agregacyjna dla danych policji
Poniżej zajmiemy się wstępną analizą dla danych policji, zobaczymy niektóre rozkłady zmiennych oraz zagregowane dane ze względu na interesujące nas cechy w bazie danych.  
Pierw ile mamy wierszy w oryginalnej ramce danych.

In [100]:
#importy
from pyspark.ml.linalg import DenseMatrix, Vectors
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import col
from pyspark.sql.functions import *

In [14]:
df_nypd.count()

1000000

In [ ]:
#sprawdzmy czy współrzędne geograficzne się zgadzają (dla NYC to 40.7128° N, 74.0060° W)

In [52]:
df_nypd.describe('latitude').show()

+-------+------------------+
|summary|          latitude|
+-------+------------------+
|  count|             99997|
|   mean| 40.73620223008082|
| stddev|0.0832308701252161|
|    min| 40.50056279200004|
|    max| 40.91272339600005|
+-------+------------------+



In [54]:
df_nypd.describe('longitude').show()

+-------+-------------------+
|summary|          longitude|
+-------+-------------------+
|  count|              99997|
|   mean| -73.92611980459544|
| stddev|0.07687619401337913|
|    min| -74.25340302899998|
|    max| -73.70072029399995|
+-------+-------------------+



In [36]:
def to_histogram_df_save(dataframe, variable):
    frame = dataframe.groupBy(variable).count()
    frame.write.format("mongodb").mode("append").save()
    return frame

In [65]:
#bazy do histogramów by zobaczyć dystrybucje danych kategorycznych
nypd_station_distr = to_histogram_df_save(df_nypd,"station_name")
nypd_susage_distr = to_histogram_df_save(df_nypd,"susp_age_group")
nypd_susrace_distr = to_histogram_df_save(df_nypd,"susp_race")
nypd_vicage_distr = to_histogram_df_save(df_nypd,"vic_age_group")
nypd_vicrace_distr = to_histogram_df_save(df_nypd,"vic_race")
nypd_vicsex_distr = to_histogram_df_save(df_nypd,"vic_sex")
nypd_boro_distr = to_histogram_df_save(df_nypd,"boro_nm")
nypd_ofns_distr = to_histogram_df_save(df_nypd,"ofns_desc")
nypd_rptdt_distr = to_histogram_df_save(df_nypd,"rpt_dt")

In [ ]:
#korelacje pomiedzy zmiennymi
#ogólnie zostawiam jak na razie bo te funkcje nie dzialaja najlepiej
#Correlation.corr(df_nypd,"susp_age_group","susp_race","vic_race","vic_sex","vic_age_group","boro_nm","ofns_desc")

### Bardziej dokładna analiza demograficzna (osobno dla napastnika oraz ofiar)
Dla kontekstu, z oficjalnych danych zamieszczonych przez zarząd miasta Nowego Yorku (dane z 2000, uwzględiają wyłącznie stale zamieszkałych) dystrybucja rasowa nowego Yorku, przy populacji 8,008,278, wyosi, w procentach,:
   - Biali - 44.7	
   - Afro-amerykanie - 26.6
   - Azjaci - 9.8
   - Latynosi - 27.0
   - Rasa mieszana - 4.9
   - Inna - 14.0

In [86]:
#agregacje dla rasy, przedziału wiekowego, płci, dystryktu gdzie zaszedł incydent oraz jego typ (skala powagi) dla ofiar 
df_sub_vic = df_nypd.select(col('vic_race'),col('vic_age_group'),col('vic_sex'),col('boro_nm'),col("law_cat_cd"))
df_vic = df_sub_vic.groupBy('vic_race', 'vic_age_group', 'vic_sex', 'boro_nm',"law_cat_cd").count()
df_vic.sort(col('count').desc()).show()

+--------------+-------------+-------+---------+-----------+-----+
|      vic_race|vic_age_group|vic_sex|  boro_nm| law_cat_cd|count|
+--------------+-------------+-------+---------+-----------+-----+
|       UNKNOWN|      UNKNOWN|      D|MANHATTAN|MISDEMEANOR| 3960|
|       UNKNOWN|      UNKNOWN|      D| BROOKLYN|MISDEMEANOR| 2447|
|       UNKNOWN|      UNKNOWN|      E|    BRONX|MISDEMEANOR| 2156|
|       UNKNOWN|      UNKNOWN|      E|MANHATTAN|MISDEMEANOR| 2072|
|       UNKNOWN|      UNKNOWN|      E| BROOKLYN|MISDEMEANOR| 2067|
|       UNKNOWN|      UNKNOWN|      D|   QUEENS|MISDEMEANOR| 1717|
|       UNKNOWN|      UNKNOWN|      D|    BRONX|MISDEMEANOR| 1503|
|       UNKNOWN|      UNKNOWN|      D|MANHATTAN|     FELONY| 1466|
|       UNKNOWN|      UNKNOWN|      E| BROOKLYN|     FELONY| 1430|
|         BLACK|        25-44|      F| BROOKLYN|MISDEMEANOR| 1341|
|       UNKNOWN|      UNKNOWN|      E|   QUEENS|MISDEMEANOR| 1032|
|       UNKNOWN|      UNKNOWN|      D| BROOKLYN|     FELONY|  

In [87]:
#agregacje dla rasy, przedziału wiekowego oraz płci dystryktu gdzie zaszedł incydent oraz jego typ (skala powagi) dla napastników
df_sub_sus = df_nypd.select(col('susp_race'),col('susp_age_group'),col('susp_sex'),col('boro_nm'),col("law_cat_cd"))
df_sus = df_sub_sus.groupBy('susp_race', 'susp_age_group', 'susp_sex','boro_nm',"law_cat_cd").count()
df_sus.sort(col('count').desc()).show()

+---------+--------------+--------+---------+-----------+-----+
|susp_race|susp_age_group|susp_sex|  boro_nm| law_cat_cd|count|
+---------+--------------+--------+---------+-----------+-----+
|     null|          null|    null| BROOKLYN|MISDEMEANOR| 4696|
|     null|          null|    null|MANHATTAN|MISDEMEANOR| 3409|
|     null|          null|    null|    BRONX|MISDEMEANOR| 3297|
|     null|          null|    null| BROOKLYN|     FELONY| 3026|
|     null|          null|    null|   QUEENS|MISDEMEANOR| 3022|
|     null|          null|    null|MANHATTAN|     FELONY| 2524|
|  UNKNOWN|       UNKNOWN|       U| BROOKLYN|MISDEMEANOR| 2217|
|     null|          null|    null|   QUEENS|     FELONY| 1988|
|  UNKNOWN|       UNKNOWN|       U|MANHATTAN|MISDEMEANOR| 1906|
|     null|          null|    null|    BRONX|     FELONY| 1638|
|    BLACK|         25-44|       M| BROOKLYN|MISDEMEANOR| 1475|
|  UNKNOWN|       UNKNOWN|       U|MANHATTAN|     FELONY| 1455|
|  UNKNOWN|       UNKNOWN|       U| BROO

### Agregacja dla czasu występowania danego przestępstwa
Dla poniższej tabeli zagregowanej będziemy przetrzymywali datę zajścia (dane zbierane od 2006 roku) od danych charakterystycznych dla danego zgłoszonego incydentu.

In [121]:
#Dane zagregowane dla daty, nazwy dzielnicy, opisu ataku, opisu okolicy, czy atak dobiegł do skutku (jego finalny stan),
#By .show sie odpaliło bez błędu należy ustawić w trakcie tworzenia sesji nowe parametry (dodane jako .set)
df_sub_time = df_nypd.select(col('rpt_dt'),col('boro_nm'),col('ofns_desc'),col('pd_desc'),col("law_cat_cd"),col("crm_atpt_cptd_cd"),
                           col("prem_typ_desc"), date_format(col("cmplnt_fr_ts"), "HH:mm:ss").alias("timestamp") )
df_time = df_sub_time.groupBy('rpt_dt', 'boro_nm', 'ofns_desc','pd_desc',"law_cat_cd","crm_atpt_cptd_cd","prem_typ_desc").agg({"timestamp":'avg',"timestamp":"min","timestamp":"max","timestamp":"count"})
df_time.sort(col('rpt_dt').asc()).show()

Unexpected exception formatting exception. Falling back to standard exception


23/01/08 12:24:31 ERROR executor.Executor: Exception in task 0.0 in stage 203.0 (TID 12337)
org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: reading dates before 1582-10-15 or timestamps before 1900-01-01T00:00:00Z from Parquet files can be ambiguous, as the files may be written by Spark 2.x or legacy versions of Hive, which uses a legacy hybrid calendar that is different from Spark 3.0+'s Proleptic Gregorian calendar. See more details in SPARK-31404. You can set spark.sql.legacy.parquet.datetimeRebaseModeInRead to 'LEGACY' to rebase the datetime values w.r.t. the calendar difference during reading. Or set spark.sql.legacy.parquet.datetimeRebaseModeInRead to 'CORRECTED' to read the datetime values as it is.
	at org.apache.spark.sql.execution.datasources.DataSourceUtils$.newRebaseExceptionInRead(DataSourceUtils.scala:147)
	at org.apache.spark.sql.execution.datasources.DataSourceUtils.newRebaseExceptionInRead(DataSourceUtils.scala)

## Wstępna analiza agregacyjna dla danych mieszkaniowych
Na początek przeprowadzimy proste agregacje dla danych mieszkaniowych by zobaczyć rozkłady dla (ważniejszych i bardziej interesujących) zmiennych.

In [123]:
df_condo.printSchema()

root
 |-- condo_section: string (nullable = true)
 |-- boro_block_lot: string (nullable = true)
 |-- address: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- building_classification: string (nullable = true)
 |-- total_units: integer (nullable = true)
 |-- year_built: integer (nullable = true)
 |-- gross_sqft: integer (nullable = true)
 |-- estimated_gross_income: integer (nullable = true)
 |-- gross_income_per_sqft: double (nullable = true)
 |-- estimated_expense: integer (nullable = true)
 |-- expense_per_sqft: double (nullable = true)
 |-- net_operating_income: integer (nullable = true)
 |-- full_market_value: integer (nullable = true)
 |-- market_value_per_sqft: double (nullable = true)
 |-- report_year: integer (nullable = true)



In [124]:
#bazy do histogramów by zobaczyć dystrybucje danych kategorycznych
condo_address = to_histogram_df_save(df_condo,"address")
condo_neighborhood = to_histogram_df_save(df_condo,"neighborhood")
condo_building_classification = to_histogram_df_save(df_condo,"building_classification")
condo_year_built = to_histogram_df_save(df_condo,"year_built")
condo_report_year = to_histogram_df_save(df_condo,"report_year")

### Dokonanie agregacji ze względu na geografie (położenie w NYC)
Poniżej dokonamy bardziej złożonej agregacji, w której dla pogrupowanych danych geograficznych oraz daty obliczamy średnie oraz
odchylenia standardowe dla danych ekonomicznych.

In [130]:
df_condo_sub = df_condo.select(col('neighborhood'),col('address'),col('building_classification')
                               ,col('report_year'),col("gross_income_per_sqft"),col("expense_per_sqft")
                               ,col("net_operating_income"), col("market_value_per_sqft"))
df_geo_condo = df_condo_sub.groupBy('neighborhood', 'address', 'building_classification','report_year').agg({"gross_income_per_sqft":'avg',"expense_per_sqft":"avg",
                                          "net_operating_income":"avg","market_value_per_sqft":"avg"})
df_geo_condo.sort(col('report_year').asc()).show(5)

+--------------------+------------------+-----------------------+-----------+-------------------------+--------------------------+---------------------+--------------------------+
|        neighborhood|           address|building_classification|report_year|avg(net_operating_income)|avg(market_value_per_sqft)|avg(expense_per_sqft)|avg(gross_income_per_sqft)|
+--------------------+------------------+-----------------------+-----------+-------------------------+--------------------------+---------------------+--------------------------+
|       ALPHABET CITY| 182 EAST 2 STREET|         R2-CONDOMINIUM|       2012|                 953656.0|                    196.63|                10.32|                     37.06|
|      HARLEM-CENTRAL|23 EAST 128 STREET|         R4-CONDOMINIUM|       2012|                 145440.0|                      86.5|                 6.81|                     18.93|
|        EAST VILLAGE|  24 EAST 7 STREET|         R9-CONDOMINIUM|       2012|                 787799

In [136]:
#Ponieważ może zostać nie użyte (i dla przejrzystości), tutaj taka sama agregacja jak powyżej tylko z sumą
#jako funkcją agregacyjną
df_condo_sub_sd = df_condo.select(col('neighborhood'),col('address'),col('building_classification')
                               ,col('report_year'),col("gross_income_per_sqft"),col("expense_per_sqft")
                               ,col("net_operating_income"), col("market_value_per_sqft"))
df_geo_condo_sd = df_condo_sub_sd.groupBy('neighborhood', 'address', 'building_classification','report_year').agg({"gross_income_per_sqft":'stddev_samp',"expense_per_sqft":"stddev_samp",
                                          "net_operating_income":"stddev_samp","market_value_per_sqft":"stddev_samp"})
df_geo_condo_sd.sort(col('report_year').desc()).show(5)

+--------------------+----------------+-----------------------+-----------+-------------------------+--------------------------+---------------------+--------------------------+
|        neighborhood|         address|building_classification|report_year|sum(net_operating_income)|sum(market_value_per_sqft)|sum(expense_per_sqft)|sum(gross_income_per_sqft)|
+--------------------+----------------+-----------------------+-----------+-------------------------+--------------------------+---------------------+--------------------------+
|           FINANCIAL|  18 PINE STREET|          R4  -ELEVATOR|       2021|                 19735687|                    290.45|                16.92|                     52.87|
|UPPER WEST SIDE (...|8 WEST 65 STREET|          R4  -ELEVATOR|       2021|                   307402|                    267.81|                11.27|                      44.2|
|        CIVIC CENTER|85 WEST BROADWAY|          R4  -ELEVATOR|       2021|                   580864|         

### Agregacja uwzględniająca rok budowy dla danego condominium
Podobna analiza jak powyżej, jednakże bierze pod uwagę głównie rok budowy oraz geografię.


In [137]:
df_condo_year = df_condo.select(col("year_built"), col('neighborhood'),col('address'),col('building_classification')
                               ,col("gross_income_per_sqft"),col("expense_per_sqft")
                               ,col("net_operating_income"), col("market_value_per_sqft"))
df_sub_year = df_condo_year.groupBy("year_built", 'neighborhood', 'building_classification').agg({"gross_income_per_sqft":'avg',"expense_per_sqft":"avg",
                                          "net_operating_income":"avg","market_value_per_sqft":"avg"})
df_sub_year.sort(col('year_built').desc()).show(5)

+----------+-----------------+-----------------------+-------------------------+--------------------------+---------------------+--------------------------+
|year_built|     neighborhood|building_classification|avg(net_operating_income)|avg(market_value_per_sqft)|avg(expense_per_sqft)|avg(gross_income_per_sqft)|
+----------+-----------------+-----------------------+-------------------------+--------------------------+---------------------+--------------------------+
|      2020|          JAMAICA|           RR  -CONRENT|                1932993.0|                     99.29|                12.58|                     27.95|
|      2020|   FLUSHING-NORTH|          R4  -ELEVATOR|                 604356.0|                    214.77|                 8.79|                     35.15|
|      2020|    CROWN HEIGHTS|          R4  -ELEVATOR|                 525028.0|                    187.01|                14.02|                     36.96|
|      2020|        GRAVESEND|          R4  -ELEVATOR|    

## Zestawienie obydwu ramek danych
Dane należało pierw poddac obróbce i zrzutować dane Nowo-Yorskiej policji na mniejszą granualność (tudzież czas z dokładnością co do roku, a położenie z dokładnością co do dystryktu, adresu) by następnie móc połączyć tabele po kluczu geograficznym oraz czasowym.

In [138]:
#sprawdzenie unikalnych
df_condo.select('neighborhood').distinct().collect()

[Row(neighborhood='MORNINGSIDE HEIGHTS'),
 Row(neighborhood='COBBLE\xa0HILL'),
 Row(neighborhood='MANOR HEIGHTS'),
 Row(neighborhood='FLATBUSH-NORTH'),
 Row(neighborhood='KIPS BAY'),
 Row(neighborhood='CYPRESS HILLS'),
 Row(neighborhood='WILLOWBROOK'),
 Row(neighborhood='WASHINGTON HEIGHTS LOWER'),
 Row(neighborhood='REGO\xa0PARK'),
 Row(neighborhood='CASTLE HILL/UNIONPORT'),
 Row(neighborhood='GREAT\xa0KILLS'),
 Row(neighborhood='CLOVE\xa0LAKES'),
 Row(neighborhood='MORNINGSIDE\xa0HEIGHTS'),
 Row(neighborhood='CASTLE\xa0HILL/UNIONPORT'),
 Row(neighborhood='ALPHABET CITY'),
 Row(neighborhood='BENSONHURST'),
 Row(neighborhood='COLLEGE\xa0POINT'),
 Row(neighborhood='CORONA'),
 Row(neighborhood='BOROUGH PARK'),
 Row(neighborhood='WINDSOR TERRACE'),
 Row(neighborhood='EAST\xa0VILLAGE'),
 Row(neighborhood='GREENWICH\xa0VILLAGE-WEST'),
 Row(neighborhood='JAMAICA\xa0HILLS'),
 Row(neighborhood='PELHAM PARKWAY NORTH'),
 Row(neighborhood='OCEAN PARKWAY-NORTH'),
 Row(neighborhood='GRYMES HILL'),


In [139]:
#sprawdzenie unikalnych
df_condo.select('address').distinct().collect()

[Row(address='507 EAST 80 STREET'),
 Row(address='75 ALLEN STREET'),
 Row(address='53 WEST 76 STREET'),
 Row(address='406 WEST 45 STREET'),
 Row(address='87 ELIZABETH STREET'),
 Row(address='295 LAFAYETTE STREET'),
 Row(address='330 PROSPECT PLACE'),
 Row(address='263 CLASSON AVENUE'),
 Row(address='2964 BRIGHTON 6 STREET'),
 Row(address='120 NASSAU STREET'),
 Row(address='141-18 CHERRY AVENUE'),
 Row(address='138-35 39 AVENUE'),
 Row(address='7\xa0LISPENARD\xa0STREET'),
 Row(address='10\xa0EAST\xa0102\xa0STREET'),
 Row(address='2040\xa0FREDERICK\xa0DOUGLASS\xa0B'),
 Row(address='2264\xa0MORRIS\xa0AVENUE'),
 Row(address='8794\xa017\xa0AVENUE'),
 Row(address='38-01\xa031\xa0AVENUE'),
 Row(address='143-36\xa037\xa0AVENUE'),
 Row(address='56 CLOVER PLACE'),
 Row(address='1 COENTIES SLIP'),
 Row(address='279 CENTRAL PARK WEST'),
 Row(address='524 EAST 72 STREET'),
 Row(address='301 STREET'),
 Row(address='19 80 STREET'),
 Row(address='139 34 AVENUE'),
 Row(address='6002 FT HAMILTON PARKWAY

In [155]:
!pip install pyspark-pandas

Defaulting to user installation because normal site-packages is not writeable


In [169]:
#rzutowanie dla danych nowego yorku
df_trans = df_nypd
df_trans = df_nypd.withColumn("Year",year("rpt_dt"))
df_trans.select(col("Year")).show()

+----+
|Year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2013|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 20 rows



In [145]:
df_trans.select(year(col('rpt_dt'))).show()

+------------+
|year(rpt_dt)|
+------------+
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
|        2013|
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
|        2018|
+------------+
only showing top 20 rows



## Wizualizacja i analiza danych w notatniku (z bazy z zagregowanymi danymi)

In [24]:
#wczytanie narzędzi do wizualizacji
import matplotlib as plt
import pandas as pd
import numpy as np

In [38]:
#bazy danych w mogodb
df = spark.read.format("mongodb").load()
df.show()

+--------------------+-------+-----+---------+--------------------+--------------+---------+-------------+--------+-------+----+
|                 _id|boro_nm|count|ofns_desc|        station_name|susp_age_group|susp_race|vic_age_group|vic_race|vic_sex|   x|
+--------------------+-------+-----+---------+--------------------+--------------+---------+-------------+--------+-------+----+
|63ba089c029e8a4bf...|   null| null|     null|                null|          null|     null|         null|    null|   null| 1.0|
|63ba7bc10fb636657...|   null|    3|     null|      148 ST.-HARLEM|          null|     null|         null|    null|   null|null|
|63ba7bc10fb636657...|   null|    5|     null|        COURT SQUARE|          null|     null|         null|    null|   null|null|
|63ba7bc10fb636657...|   null|    4|     null|       MOSHOLU PKWY.|          null|     null|         null|    null|   null|null|
|63ba7bc10fb636657...|   null|    6|     null|VERNON BLVD.-JACK...|          null|     null|     